In [1]:
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import time
import pandas as pd
import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import cohere
import numpy as np
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


In [2]:
MY_JSON = r"C:\Users\Sparsh Mahajan\Downloads\famous-badge-422913-k5-86385cd35b0b.json"
co = cohere.Client("r4TYW3pued8wTV0CC8dEYnIoiCPN5MyCBn0ecDej")

In [3]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name(MY_JSON, scope)

# authorize the clientsheet 
client = gspread.authorize(creds)

In [4]:
sheet = client.open_by_url("https://docs.google.com/spreadsheets/d/1ACJR6Acf9DUuWvp13kwZOjmJCaogZsU7XVbAXQ7Wfh4/edit#gid=0").sheet1


In [5]:
def create_df(url , sheet_number):
    """
  Take google sheet url and convert it to dataframe.

  Args:
      url: string containing url of google sheet of given user.
      sheet_number : Number of sheet which will be used

  Returns:
      sheet_df : Pandas Dataframe containing all the Parameters in sheet.

  Raises:
      TypeError: Problem with url or data is null.
  """
    try:
        sheet = client.open_by_url(url) ## search by url 
    except:
        raise requests.exceptions.RequestException(f"url not found")  ## if url not found
            
            
    worksheet = sheet.get_worksheet(sheet_number) ## search for sheet
    
    if not worksheet:
          raise FileNotFoundError(f"Given sheet is empty")
            
    sheet_df= pd.DataFrame.from_dict(worksheet.get_all_records())
    
    return sheet_df
            
    
            
    
    

    
    

In [6]:
sheet_df = create_df("https://docs.google.com/spreadsheets/d/1mm0zLo7SR2wN1Zq1K2Vfld6lQGZ_bbX2kbj6owhC2-c/edit#gid=0" , 0)

In [7]:
sheet_df

,Company Name,companywebsite
0,WictroniX,https://wictronix.com/
1,Tomaque Design Labs,https://tomaque.com
2,The Thirsty Crow Marketing Services,http://www.ttc-marketing.com
3,Evolutyz Corp,https://www.evolutyz.com
4,LeadTronics,http://www.lead-tronics.com
5,Blue Meteor,https://amazepxm.com/
6,Techy Mau Game Studios,https://techymau.games
7,Saskia Tech,https://www.saskiatech.com/
8,Cloudlytics,https://www.cloudlytics.com/
9,Desk365,https://www.desk365.io/


In [8]:
def clean_scrapedtext(html):
    """
       Remove useless elements from the BeautifulSoup object.
       useless elements include : 
       1. Script 
       2. Style
       3. Header
       4. Footer
       5. Advertisements
       6. Navigation elements

      Args:
          HTML: HTML content of page.

      Returns:
          Final Cleaned text
    """
   
    soup = BeautifulSoup(html , "html.parser")
    
    ad_class = soup.find_all('div', class_=lambda class_: class_ in ['ad-container', 'advertisement' , 'adv' , 'ad' , 'sponsored' , 'ads' , 'ul' , 'li' , 'nav']) #elements with specific adv class names
    ad_atr = soup.find_all('div', attrs={'data-ad-slot': True }) # Target elements with data attributes
    ads = ad_class + ad_atr # Combine identified ad elements
     
    for data in soup(['style', 'script' , 'header' , 'footer' , 'nav' , 'li' , 'ul']) or ads: ##removing script styles header footer and ads from website 
        data.decompose()

    return ' '.join(soup.stripped_strings)

In [9]:
def scrape(URL):
    """
    Scrape URL of given Website.
    Args:
        URL : url of website.
    Return:
        NO ERROR : Cleaned text after scraping.
        ERROR: Empty String.
         """
    try:
        page = requests.get(URL)  ## getting url and waiting for some time so that it would load
        time.sleep(2)
        return clean_scrapedtext(page.content)
    except:
        return  "" ## return empty string if error
    
    

In [10]:
def new_col(df , col_name):
    """
    update sheet by adding scraped data from each url
    Args :
       df = dataframe
       col_name = column name with url
    Returns:
    """
    lst = []  ##list to store values
    i=0;
    for url in df[col_name]:
        print(f"scraping {i} website")
        i+=1
        lst.append(scrape(url)) ## scraping urls
        
    ##updating dataframe
    df['Scraped and formated homepage data'] = lst  
    
    
    

In [11]:
## Optional Step  : to reduce number of count of tokens while retaining accuracy of prompt might affect accuracy of result



def preprocess(text):
    """reducing size of column using nlp preprocessing technique
    Args:
        text: text to be preprocessed
        
    returns:
      clean_text : Preprocessed_text
    """
    
    text = text.lower()  ##lowering
 
    word_tokens = nltk.word_tokenize(text) ##token
    
    filtered_sentence=[]
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(lemmatizer.lemmatize(w))  ## stop word removal and lemmatization
            
    return " ".join(filtered_sentence)
    
    

In [12]:
new_col(sheet_df , "companywebsite" )

scraping 0 website
scraping 1 website
scraping 2 website
scraping 3 website
scraping 4 website
scraping 5 website
scraping 6 website
scraping 7 website
scraping 8 website
scraping 9 website
scraping 10 website
scraping 11 website
scraping 12 website
scraping 13 website
scraping 14 website
scraping 15 website
scraping 16 website
scraping 17 website
scraping 18 website
scraping 19 website
scraping 20 website
scraping 21 website
scraping 22 website
scraping 23 website
scraping 24 website
scraping 25 website
scraping 26 website
scraping 27 website
scraping 28 website
scraping 29 website
scraping 30 website
scraping 31 website
scraping 32 website
scraping 33 website
scraping 34 website
scraping 35 website
scraping 36 website
scraping 37 website
scraping 38 website
scraping 39 website
scraping 40 website
scraping 41 website
scraping 42 website
scraping 43 website
scraping 44 website
scraping 45 website


In [13]:
sheet_df["Scraped and formated homepage data"][0]

"WictroniX - Integration Is Incredible Integration Is Incredible What do we do ? Identify Problems -> Research and Analysis -> Strategy Creation -> Execution -> Feedback We will give you a solid and effective execution as the backbone of your startup. In order to begin your journey from good to great, integrate with WictroniX. Let us demonstrate how we may assist: Our Magical Spells The X -Factor Performing routine tasks does not make a business exceptional. You must possess a differentiating quality that makes you stand out from your rivals. Here at Wictronix, we always concentrate on looking for what we refer to as the X-Factor. Additionally, it appears right here in our name. Integrated e X ecution There is always friction and inefficiency if different types of tasks are assigned to various authorities.Without any hassle, we set up a precise and correct framework underneath to enable you to focus on your product. Speed and Efficiency No company wants to be sluggish in todays quick-p

# Prompting

In [14]:
## cohere api to get answer for prompt using scraped data
## create list of it
## update it in df

def prompt1(prompt , col , df):
    """
    Add new column give result of prompt1 using df and cohere
    
    Args:
        prompt: prompt used
        col :column name of created column
        df : dataframe
    
    
    """
    lst = []  ##list to store values
    for d in df['Scraped and formated homepage data']:
        if d!="": ## is d is empty then threre is no need to generate text
            txt = prompt.replace("INSERT Scraped and cleaned information" , d)  ##replacing string to get value
            text = co.chat(message=txt).text ## cohere api 
        else:
            text=""
        lst.append(text) ## appending
        
    df[col] = lst
    
    
    
def prompt2(prompt , col , df):
    """
    Add new column give result of prompt 2  using df and cohere  and result of prompt 1
    
    Args:
        prompt: prompt used
        col :column name of created column
        df : dataframe
    
    
    """
    lst = [] ##list to store values
    for d in df['Output from Prompt 1']:
        if d!="": ## is d is empty then threre is no need to generate text
            txt = prompt.replace("Insert output from prompt 1" , d ) ##replacing string to get value
            text = co.chat(message = txt).text
        else:
            text=""
        lst.append(text) ## appending
    df[col] = lst 
    

    
    

In [15]:
def prompt3(prompt , col , df):
    """
    Add new column give result of prompt 3 using df and cohere and prompt 1 and 2 
    
    Args:
        prompt: prompt used
        col :column name of created column
        df : dataframe
    
    
    """
        
    lst = [] ##list to store values
    n = len(df) ## n length of dataframe
    for i in range(n): 
        if df['Scraped and formated homepage data'][i] !='':
            txt= prompt.replace("Insert output from prompt 1" ,  df['Output from Prompt 1'][i] )
            txt= txt.replace("Insert output from prompt 2" ,  df['Output from Prompt 2'][i] )
            text = co.chat(message = txt).text
        else:
            text = ""
            
        lst.append(text) ## appending
        time.sleep(10)
            
    df[col] = lst ## changing

In [16]:
prompt = client.open_by_url("https://docs.google.com/spreadsheets/d/1mm0zLo7SR2wN1Zq1K2Vfld6lQGZ_bbX2kbj6owhC2-c/edit#gid=0")
worksheet = prompt.get_worksheet(1)
prompts = np.array(worksheet.get_all_values()[1:])


In [17]:
df= sheet_df

In [18]:
prompt1(prompts[0][0] , 'Output from Prompt 1' , df)
df

,Company Name,companywebsite,Scraped and formated homepage data,Output from Prompt 1
0,WictroniX,https://wictronix.com/,WictroniX - Integration Is Incredible Integrat...,## Company Overview: \nWictroniX is a business...
1,Tomaque Design Labs,https://tomaque.com,Full Service Digital Marketing Agency- Tomaque...,Company Overview: \nTomaque Digital Services i...
2,The Thirsty Crow Marketing Services,http://www.ttc-marketing.com,Not Acceptable! Not Acceptable! An appropriate...,"I apologize, but the information you provided ..."
3,Evolutyz Corp,https://www.evolutyz.com,Top IT Consulting Firms in Chicago ǀ Tech Cons...,Here is a summary of the provided homepage con...
4,LeadTronics,http://www.lead-tronics.com,,
5,Blue Meteor,https://amazepxm.com/,Amaze PXM - Product Experience Management for ...,Here is a summary of the provided homepage con...
6,Techy Mau Game Studios,https://techymau.games,Techy Mau Game Studios - Leading Game Studio f...,## Company Overview: \nTechy Mau Game Studios ...
7,Saskia Tech,https://www.saskiatech.com/,,
8,Cloudlytics,https://www.cloudlytics.com/,Cloudlytics - Cloud-Driven Security for Modern...,Company Overview: \nCloudlytics is a trusted s...
9,Desk365,https://www.desk365.io/,Microsoft Teams Ticketing System | Desk365 Men...,Here is a summary of the provided homepage con...


In [19]:
prompt2(prompts[0][1] , 'Output from Prompt 2' , df)
df

,Company Name,companywebsite,Scraped and formated homepage data,Output from Prompt 1,Output from Prompt 2
0,WictroniX,https://wictronix.com/,WictroniX - Integration Is Incredible Integrat...,## Company Overview: \nWictroniX is a business...,Here is a potential sales pitch for your custo...
1,Tomaque Design Labs,https://tomaque.com,Full Service Digital Marketing Agency- Tomaque...,Company Overview: \nTomaque Digital Services i...,Here is a potential sales pitch for your custo...
2,The Thirsty Crow Marketing Services,http://www.ttc-marketing.com,Not Acceptable! Not Acceptable! An appropriate...,"I apologize, but the information you provided ...","I apologize, but it seems the company overview..."
3,Evolutyz Corp,https://www.evolutyz.com,Top IT Consulting Firms in Chicago ǀ Tech Cons...,Here is a summary of the provided homepage con...,Here is a potential sales opportunity analysis...
4,LeadTronics,http://www.lead-tronics.com,,,
5,Blue Meteor,https://amazepxm.com/,Amaze PXM - Product Experience Management for ...,Here is a summary of the provided homepage con...,# Potential Sales Opportunity: \n\n## Target C...
6,Techy Mau Game Studios,https://techymau.games,Techy Mau Game Studios - Leading Game Studio f...,## Company Overview: \nTechy Mau Game Studios ...,Here is a potential sales opportunity for your...
7,Saskia Tech,https://www.saskiatech.com/,,,
8,Cloudlytics,https://www.cloudlytics.com/,Cloudlytics - Cloud-Driven Security for Modern...,Company Overview: \nCloudlytics is a trusted s...,Here is a potential sales opportunity for your...
9,Desk365,https://www.desk365.io/,Microsoft Teams Ticketing System | Desk365 Men...,Here is a summary of the provided homepage con...,Here is an analysis of the potential sales opp...


In [20]:
prompt3(prompts[0][2] , 'Output from Prompt 3' , df)
df

,Company Name,companywebsite,Scraped and formated homepage data,Output from Prompt 1,Output from Prompt 2,Output from Prompt 3
0,WictroniX,https://wictronix.com/,WictroniX - Integration Is Incredible Integrat...,## Company Overview: \nWictroniX is a business...,Here is a potential sales pitch for your custo...,"Dear [Client], \n\nI wanted to reach out and i..."
1,Tomaque Design Labs,https://tomaque.com,Full Service Digital Marketing Agency- Tomaque...,Company Overview: \nTomaque Digital Services i...,Here is a potential sales pitch for your custo...,"Dear [Client], \n\nI wanted to reach out and i..."
2,The Thirsty Crow Marketing Services,http://www.ttc-marketing.com,Not Acceptable! Not Acceptable! An appropriate...,"I apologize, but the information you provided ...","I apologize, but it seems the company overview...","Dear [Client], \n\nI wanted to reach out and i..."
3,Evolutyz Corp,https://www.evolutyz.com,Top IT Consulting Firms in Chicago ǀ Tech Cons...,Here is a summary of the provided homepage con...,Here is a potential sales opportunity analysis...,"Dear [Client], \n\nI wanted to reach out and i..."
4,LeadTronics,http://www.lead-tronics.com,,,,
5,Blue Meteor,https://amazepxm.com/,Amaze PXM - Product Experience Management for ...,Here is a summary of the provided homepage con...,# Potential Sales Opportunity: \n\n## Target C...,"Dear [Client], \n\nI wanted to reach out and i..."
6,Techy Mau Game Studios,https://techymau.games,Techy Mau Game Studios - Leading Game Studio f...,## Company Overview: \nTechy Mau Game Studios ...,Here is a potential sales opportunity for your...,"Dear [Client], \n\nI wanted to reach out and i..."
7,Saskia Tech,https://www.saskiatech.com/,,,,
8,Cloudlytics,https://www.cloudlytics.com/,Cloudlytics - Cloud-Driven Security for Modern...,Company Overview: \nCloudlytics is a trusted s...,Here is a potential sales opportunity for your...,"Dear [Client], \n\nI wanted to reach out and i..."
9,Desk365,https://www.desk365.io/,Microsoft Teams Ticketing System | Desk365 Men...,Here is a summary of the provided homepage con...,Here is an analysis of the potential sales opp...,"Dear [Client], \n\nI wanted to reach out and i..."


In [21]:
sheet.update([df.columns.values.tolist()] + df.values.tolist())

C:\conda_tmp\ipykernel_36056\284927942.py:1: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  sheet.update([df.columns.values.tolist()] + df.values.tolist())


{'spreadsheetId': '1ACJR6Acf9DUuWvp13kwZOjmJCaogZsU7XVbAXQ7Wfh4',
 'updatedRange': "'Target Company Data'!A1:F47",
 'updatedRows': 47,
 'updatedColumns': 6,
 'updatedCells': 282}

In [32]:
def workflow(url , sheet_number , col_name , prompt_sheet_number):
    """
    workflow of whole process created using all the code based upon given requirements.
    
    Args:
        url : url of spreadsheet
        sheet_number  : number of sheet to be scraped
        col_name : name pf column to be scrapped
        prompt_sheet_number :  number of prompt sheet
    
    """
    df = create_df(url , sheet_number)
    new_col(df , col_name)
    ##updation 1
    ## sheet.update([df.columns.values.tolist()] + df.values.tolist()) ## since we are updating at the end it serves no purpose 
    
    ##optional 
    ## df['Scraped and formated homepage data'] = df[col_name].apply(lambda x : preprocess(x))
    
    prompt = client.open_by_url(url)
    worksheet = prompt.get_worksheet(prompt_sheet_number)
    prompts = np.array(worksheet.get_all_values()[1:])
    
    
    prompt1(prompts[0][0] , 'Output from Prompt 1' , df)
    prompt2(prompts[0][1] , 'Output from Prompt 2' , df)
    prompt3(prompts[0][2] , 'Output from Prompt 3' , df)
    
    
    sheet.update([df.columns.values.tolist()] + df.values.tolist())
    
    
    
def final_automation(url , json , client_key , sheet_number , col_name , prompt_sheet_number):
    """
    authorise user and do workflow
    
    Args:
        url : url of google sheet
        json : google sheet service account key json format
        client_key = cohere api key
    """
    
    MY_JSON = json
    co = cohere.Client(client_key)
    
    
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

    # add credentials to the account
    creds = ServiceAccountCredentials.from_json_keyfile_name(MY_JSON, scope)

    # authorize the clientsheet 
    client = gspread.authorize(creds)
    
    workflow(url , sheet_number , col_name , prompt_sheet_number)
    
 

url = "https://docs.google.com/spreadsheets/d/1mm0zLo7SR2wN1Zq1K2Vfld6lQGZ_bbX2kbj6owhC2-c/edit#gid=0"
json = r"C:\Users\Sparsh Mahajan\Downloads\famous-badge-422913-k5-86385cd35b0b.json"
client_key = "r4TYW3pued8wTV0CC8dEYnIoiCPN5MyCBn0ecDej"
sheet_number = 0
col_name="companywebsite"
prompt_sheet_number = 1

final_automation(url , json , client_key , sheet_number , col_name , prompt_sheet_number)

C:\conda_tmp\ipykernel_17140\284927942.py:1: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  sheet.update([df.columns.values.tolist()] + df.values.tolist())


{'spreadsheetId': '1ACJR6Acf9DUuWvp13kwZOjmJCaogZsU7XVbAXQ7Wfh4',
 'updatedRange': "'Target Company Data'!A1:F47",
 'updatedRows': 47,
 'updatedColumns': 6,
 'updatedCells': 282}